# Slabs of SrTiO3(011) with specified terminations

## 0. Introduction

This notebook demonstrates how to create slabs of SrTiO3(011) with specified terminations following the manuscript:

> **R. I. Eglitis and David Vanderbilt**
> *First-principles calculations of atomic and electronic structure of SrTiO3 (001) and (011) surfaces*
> Phys. Rev. B 77, 195408 (2008)
> [DOI: 10.1103/PhysRevB.77.195408](https://doi.org/10.1103/PhysRevB.77.195408)

Replicating the material from the FIG. 2. with 2 different terminations:

 ![FIG. 2.](https://i.imgur.com/W3dPmWK.png)



## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:
MILLER_INDICES = (0, 1, 1)
THICKNESS = 3  # in atomic layers
VACUUM = 10.0  # in angstroms
XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
USE_ORTHOGONAL_Z = True
USE_CONVENTIONAL_CELL = True


### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples")

### 1.3. Get input materials

In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials
from mat3ra.made.tools.modify import rotate

material = Material(Materials.get_by_name_first_match("SrTiO3"))
# Rotate material to correctly identify multiple terminations.
# Without the rotation, the current implementation only finds a single termination (as of 2024-12).
material = rotate(material, axis=[1, 0, 0], angle=10)

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize(material, repetitions=[3, 3, 3], rotation="0x")
visualize(material, repetitions=[3, 3, 3], rotation="-90x")

## 2. Configure slab

### 2.1. Create slab configuration
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.


In [ ]:

from mat3ra.made.tools.build.slab import SlabConfiguration

slab_configuration = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDICES,
    thickness=THICKNESS,  # in atomic layers
    vacuum=VACUUM,  # in angstroms
    xy_supercell_matrix=XY_SUPERCELL_MATRIX,
    use_orthogonal_z=USE_ORTHOGONAL_Z,
    use_conventional_cell=USE_CONVENTIONAL_CELL,
)

### 2.2. Get possible terminations for the slabs

In [ ]:
from mat3ra.made.tools.build.slab import get_terminations

slab_terminations = get_terminations(slab_configuration)
print("Terminations")
for idx, termination in enumerate(slab_terminations):
    print(f"    {idx}: {termination}")

## 3. Visualize slabs for all possible terminations

In [ ]:
from mat3ra.made.tools.build.slab import create_slab

slabs = [create_slab(slab_configuration, termination) for termination in slab_terminations]

visualize([{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in slabs], repetitions=[3, 3, 1])
visualize([{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in slabs], repetitions=[3, 3, 1],
          rotation="-90x")

## 4. Download materials

In [ ]:
from utils.jupyterlite import download_content_to_file

for slab in slabs:
    download_content_to_file(slab, f"{slab.name}.json")